You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Content Plan Document:

Title: Unveiling the Latest Trends in Artificial Intelligence

Introduction:
- Brief overview of the significance of Artificial Intelligence in today's world
- Mention of key players in the AI industry
- Preview of the key points to be covered in the article

Key Points:
1. Latest Trends in Artificial Intelligence:
- Advancements in machine learning algorithms
- Integration of AI in various industries such as healthcare, finance, and retail
- The rise of autonomous vehicles and smart a

I now can give a great answer

Final Answer: 

# Unveiling the Latest Trends in Artificial Intelligence

## Introduction
Artificial Intelligence (AI) has become an integral part of our modern world, revolutionizing industries and shaping the way we interact with technology. From advancements in machine learning algorithms to the integration of AI in sectors like healthcare, finance, and retail, the impact of AI is undeniable. Key players in the AI industry, such as Google DeepMind, IBM Watson, Amazon Web Services, Microsoft Azure, and OpenAI, are constantly pushing the boundaries of what AI can achieve. In this article, we will delve into the latest trends in AI, explore the key players driving innovation, and discuss noteworthy news in the field.

## Latest Trends in Artificial Intelligence
One of the most significant trends in AI is the continuous advancements in machine learning algorithms. These algorithms are becoming more sophisticated and efficient, allowing AI systems to learn 

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# Unveiling the Latest Trends in Artificial Intelligence

## Introduction
Artificial Intelligence (AI) has become an integral part of our modern world, revolutionizing industries and shaping the way we interact with technology. From advancements in machine learning algorithms to the integration of AI in sectors like healthcare, finance, and retail, the impact of AI is undeniable. Key players in the AI industry, such as Google DeepMind, IBM Watson, Amazon Web Services, Microsoft Azure, and OpenAI, are constantly pushing the boundaries of what AI can achieve. In this article, we will delve into the latest trends in AI, explore the key players driving innovation, and discuss noteworthy news in the field.

## Latest Trends in Artificial Intelligence
One of the most significant trends in AI is the continuous advancements in machine learning algorithms. These algorithms are becoming more sophisticated and efficient, allowing AI systems to learn and adapt in real-time. Additionally, AI is being increasingly integrated into various industries, transforming processes and driving efficiency. From healthcare diagnostics to financial forecasting, AI is revolutionizing the way businesses operate. Moreover, the rise of autonomous vehicles and smart assistants is reshaping how we interact with technology on a daily basis. However, as AI becomes more prevalent, ethical considerations in its development and deployment are gaining importance, ensuring that AI technologies are used responsibly and ethically.

## Key Players in the AI Industry
The AI industry is dominated by key players who are at the forefront of innovation. Google DeepMind, known for its groundbreaking work in deep learning and reinforcement learning, continues to push the boundaries of AI research. IBM Watson, with its cognitive computing capabilities, is revolutionizing industries like healthcare and finance. Amazon Web Services and Microsoft Azure are leading providers of AI services and solutions, enabling businesses to leverage AI technologies effectively. OpenAI, founded with the mission to ensure that artificial general intelligence benefits all of humanity, is driving research in AI ethics and safety.

## Noteworthy News in Artificial Intelligence
Recent breakthroughs in natural language processing have enabled AI systems to understand and generate human-like text, opening up new possibilities for applications in content creation and communication. AI is also being utilized to combat climate change, with innovative solutions for environmental monitoring and conservation. However, the impact of AI on job automation and workforce transformation raises concerns about the future of work. Regulatory developments in AI governance are crucial to ensure that AI technologies are developed and deployed responsibly, addressing issues of bias, fairness, and transparency.

In conclusion, Artificial Intelligence is a rapidly evolving field with profound implications for society and the economy. By staying informed about the latest trends, key players, and noteworthy news in AI, professionals, business owners, and students can navigate the complex landscape of AI technologies and contribute to shaping a more ethical and sustainable future. Stay engaged with AI developments by subscribing to industry newsletters, attending AI conferences, and sharing your thoughts on the latest advancements in the comments section below.

---
*Stay updated on Artificial Intelligence trends by subscribing to our newsletter or attending industry conferences. Share your thoughts on the latest AI developments in the comments section below. For further reading, check out our related articles and resources.*

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [13]:
topic = "Artificial General Intelligence"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial General Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan: 

Title: The Future of Artificial General Intelligence: Trends, Players, and News

Introduction: 
- Brief overview of Artificial General Intelligence (AGI) and its potential impact on society
- Mention of recent developments and breakthroughs in the field

Key Points: 
1. Latest Trends in AGI:
- Advancements in machine learning algorithms and neural networks
- Integration of AGI in various industries such as healthcare, finance, and transportation
- Ethical considerations and regulation

I now can give a great answer.

Final Answer: 

# The Future of Artificial General Intelligence: Trends, Players, and News

## Introduction

Artificial General Intelligence (AGI) has become a captivating subject in the tech industry due to its potential to revolutionize society. Recent breakthroughs in AGI have garnered the attention of researchers, industry professionals, and tech enthusiasts alike. Advances in machine learning and neural networks are continuously expanding the capabilities of AI, bringing to light the implications of AGI on various industries and the ethical considerations surrounding its development.

## Latest Trends in AGI

The field of AGI has witnessed significant progress in recent years, particularly in machine learning algorithms and neural networks. These advancements have paved the way for the incorporation of AGI in industries such as healthcare, finance, and transportation, where AI-driven solutions are reshaping operations and decision-making processes. 

In [14]:
Markdown(result)

# The Future of Artificial General Intelligence: Trends, Players, and News

## Introduction

Artificial General Intelligence (AGI) has become a captivating subject in the tech industry due to its potential to revolutionize society. Recent breakthroughs in AGI have garnered the attention of researchers, industry professionals, and tech enthusiasts alike. Advances in machine learning and neural networks are continuously expanding the capabilities of AI, bringing to light the implications of AGI on various industries and the ethical considerations surrounding its development.

## Latest Trends in AGI

The field of AGI has witnessed significant progress in recent years, particularly in machine learning algorithms and neural networks. These advancements have paved the way for the incorporation of AGI in industries such as healthcare, finance, and transportation, where AI-driven solutions are reshaping operations and decision-making processes. However, the rise of AGI also brings forth a myriad of ethical dilemmas and regulatory hurdles that must be navigated to ensure the responsible and safe deployment of these technologies.

## Key Players in AGI

At the forefront of AGI research and development are companies like OpenAI, DeepMind, and IBM, who are at the forefront of expanding the boundaries of AI capabilities and exploring new horizons in machine intelligence. The collaboration between academia and industry has played a pivotal role in the advancement of AGI technologies, with influential figures in the AGI community driving innovation and molding the future of artificial intelligence.

## Noteworthy News in AGI

Recent studies and experiments have demonstrated the remarkable abilities of AGI systems, ranging from natural language processing to complex decision-making tasks. Updates on AGI projects and initiatives worldwide continue to showcase the rapid pace of innovation in the field, prompting discussions on the impact of AGI on job markets and the necessity for workforce automation. Keeping abreast of the latest trends and developments in AGI technologies is essential for navigating the intricate landscape of AI research.

## Conclusion

As the field of Artificial General Intelligence progresses, it is essential for tech enthusiasts, researchers, students, and industry professionals to stay informed about the latest trends, key players, and significant news in AGI. By addressing ethical considerations, exploring future applications of AGI technologies, and engaging in dialogues about the societal impact of AI, we can shape a future where AGI enhances humanity in meaningful and responsible ways.

Remember to subscribe to industry newsletters, participate in AI conferences, and contribute to discussions on the future of AGI. Your insights and input are invaluable in shaping the trajectory of artificial intelligence technologies.

Let us continue to delve into the possibilities of AGI and embrace the opportunities it offers for a brighter and more intelligent future.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).